# Action and Policy
This section will discuss how to control the vehicle in MetaDrive with the *Policy* interface. Before this, let's have a look at the raw control signal required by vehicles.

## Action

To control vehicles in MetaDrive, the input should be a normalized action: $\mathbf a = [a_1, a_2] \in [-1, 1]^2$. This action is converted into the steering $u_s$ (degree), acceleration $u_a$ (hp) and brake signal $u_b$ (hp) in the following ways:

$u_s = S_{max} a_1$  
$u_a = F_{max} \max(0, a_2)$  
$u_b = -B_{max} \min(0, a_2)$  

wherein $S_{max}$ (degree) is the maximal steering angle, $F_{max}$ (hp) is the maximal engine force, and $B_{max}$ (hp) is the maximal brake force. 
To increase diversity, the accurate values of these parameters are varying across different vehicle but are limited to certain ranges defined by [VehicleParameterSpace](https://github.com/metadriverse/metadrive/blob/main/metadrive/utils/space.py).

The steering $u_s$ is applied to two front wheels. In addition, the engine force $u_a$ and the brake force $u_b$ are applied to four wheels, as the car in MetaDrive is four-wheel drive (4WD). The concrete implementation is as follows:

In [5]:
from metadrive.component.vehicle.base_vehicle import BaseVehicle
import inspect
print(inspect.getsource(BaseVehicle._set_action))
print(inspect.getsource(BaseVehicle._apply_throttle_brake))

    def _set_action(self, action):
        if action is None:
            return
        steering = action[0]
        self.throttle_brake = action[1]
        self.steering = steering
        self.system.setSteeringValue(self.steering * self.max_steering, 0)
        self.system.setSteeringValue(self.steering * self.max_steering, 1)
        self._apply_throttle_brake(action[1])

    def _apply_throttle_brake(self, throttle_brake):
        max_engine_force = self.config["max_engine_force"]
        max_brake_force = self.config["max_brake_force"]
        for wheel_index in range(4):
            if throttle_brake >= 0:
                self.system.setBrake(2.0, wheel_index)
                if self.speed_km_h > self.max_speed_km_h:
                    self.system.applyEngineForce(0.0, wheel_index)
                else:
                    self.system.applyEngineForce(max_engine_force * throttle_brake, wheel_index)
            else:
                if self.enable_reverse:
                    s

Actually, you can make the car 2WD or 4 wheel steering or even increasing its number of wheels by implementing a new vehicle type like `BaseVehicle`.

The aforementioned `_set_action(self, action)` function is wrapped by the `before_step(self, action)` function, which will do additional manipulations like numerical validation. Thus to control a vehicle, just set the action through `vehicle.before_step(target_action)` before simulating the next step. A minimal example to control a vehicle is as follows. The script first creates a new vehicle in front of the green ego car and set its action as `[0, 0.05]` at each step. As a result, it slowly moves forward, while the green ego car stops at the origin as its input action is always `[0, 0]`. 

In [ ]:
from metadrive.envs.metadrive_env import MetaDriveEnv
from metadrive.component.vehicle.vehicle_type import DefaultVehicle

env=MetaDriveEnv(dict(map="S", traffic_density=0))
try:
    env.reset()
    cfg=env.config["vehicle_config"]
    cfg["navigation"]=None # it doesn't need navigation system
    v = env.engine.spawn_object(DefaultVehicle, vehicle_config=cfg, position=[30,0], heading=0)
    for _ in range(500):
        v.before_step([0, 0.05])
        env.step([0,0])
        env.render(mode="topdown", screen_size=(800, 400),camera_position=(60,10))
finally:
    env.close()

## Policy

Generally, we want to control the vehicle in several ways besides providing raw action. For example, we may want to have an intermediate controller to turn the high-level driving commands into the low-level steering, throttle and brake signals. MetaDrive thus provides such an interface called *Policy*. The output of any *Policy* is supposed to be the raw signals taken by `vehicle.before_step` as input. To make such driving decisions, *Policy* can access all simulation information such as maps and objects as well as the external input signals from `env.step`. This allows us to make either an intermediate controller or a self-contained self-driving policy like `IDMPolicy`. Let's firstly have a look at the basic `EnvInputPolicy`. 

### EnvInputPolicy
By such design, the action space for each agent is always fixed to `gym.spaces.Box(low=-1.0, high=1.0, shape=(2, ))`. However, we provides a config named `extra_action_dim` (int) which allows user to add more dimensions in the action space.
For example, if we set `config["extra_action_dim"] = 1`, then the action space for each agent will become `Box(-1.0, 1.0, shape=(3, ))`. This allow the user to write environment wrapper that introduce more input action dimensions.